Computing bids for the hybrid model

In [2]:
import math
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint, NonlinearConstraint

df = pd.read_csv("Root_Insurance_data.csv")
#copying and pasting numbers without needing to run code again

#_name is the 4 features
_name = ['Currently Insured', 'Number of Vehicles', 'Number of Drivers', 'Marital Status']
#_index keeps track of stuff
#(currently insured, number vehicles, number drivers, marital status)
_index = [['Y', 1, 1, 'M'],
 ['Y', 2, 2, 'M'],
 ['Y', 3, 1, 'S'],
 ['Y', 1, 2, 'S'],
 ['Y', 2, 1, 'M'],
 ['Y', 3, 2, 'M'],
 ['Y', 1, 1, 'S'],
 ['Y', 2, 2, 'S'],
 ['Y', 3, 1, 'M'],
 ['Y', 1, 2, 'M'],
 ['Y', 2, 1, 'S'],
 ['Y', 3, 2, 'S'],
 ['N', 1, 1, 'M'],
 ['N', 2, 2, 'M'],
 ['N', 3, 1, 'S'],
 ['N', 1, 2, 'S'],
 ['N', 2, 1, 'M'],
 ['N', 3, 2, 'M'],
 ['N', 1, 1, 'S'],
 ['N', 2, 2, 'S'],
 ['N', 3, 1, 'M'],
 ['N', 1, 2, 'M'],
 ['N', 2, 1, 'S'],
 ['N', 3, 2, 'S'],
 ['unknown', 1, 1, 'M'],
 ['unknown', 2, 2, 'M'],
 ['unknown', 3, 1, 'S'],
 ['unknown', 1, 2, 'S'],
 ['unknown', 2, 1, 'M'],
 ['unknown', 3, 2, 'M'],
 ['unknown', 1, 1, 'S'],
 ['unknown', 2, 2, 'S'],
 ['unknown', 3, 1, 'M'],
 ['unknown', 1, 2, 'M'],
 ['unknown', 2, 1, 'S'],
 ['unknown', 3, 2, 'S']]

#click = number of clicks for each status
#sold = number of policies sold
#total = number in each category
click = [37, 21, 24, 77, 19, 18, 78, 16, 21, 76, 15, 11, 25, 33, 22, 79, 15, 21, 43, 32, 15, 74, 16, 13, 112, 94, 67, 128, 84, 73, 127, 110, 69, 105, 108, 0]
sold = [11, 8, 5, 24, 6, 2, 38, 3, 4, 24, 4, 4, 14, 15, 8, 32, 6, 11, 25, 15, 9, 31, 5, 3, 59, 26, 26, 55, 41, 32, 69, 42, 25, 57, 44, 0]
total = [274, 317, 267, 282, 284, 276, 290, 304, 294, 295, 266, 270, 300, 296, 286, 283, 263, 326, 271, 316, 265, 275, 272, 291, 294, 306, 256, 303, 268, 277, 280, 288, 287, 294, 284, 0]



#probsold = number sold/10000
#probSoldGivenClick = numbersold/number click
probsold=[]#probably sold
probSoldGivenClick = [] #prob sold given click
for i in range(0,len(click)):
    if click[i] !=0:
        probsold.append(sold[i]/10000)
        probSoldGivenClick.append(sold[i]/click[i])
    else:
        probsold.append(0)
        probSoldGivenClick.append(0)
        
#prRF = pd.read_csv('probabilities.csv')
#probRF = prRF['prob from random forests']
        
#totalrank, total number in each rank, 1 to 5
#clickperrank = clicks in each rank, 1 to 5
totalrank = [1611, 1608, 2401, 2089, 2291]
clickperrank = [848, 462, 351, 168, 49]

#dividedrankmean = average rank for each category
dividedrankmean = [3.5693430656934306, 4.302839116719243, 4.2434456928838955, 2.5106382978723403, 4.264084507042254, 4.3768115942028984, 2.5482758620689654, 4.3256578947368425, 4.316326530612245, 2.559322033898305, 4.349624060150376, 4.325925925925926, 3.5733333333333333, 3.4932432432432434, 4.255244755244755, 2.441696113074205, 4.391634980988593, 4.245398773006135, 3.4354243542435423, 3.632911392405063, 4.305660377358491, 2.4545454545454546, 4.286764705882353, 4.243986254295533, 1.6904761904761905, 1.65359477124183, 2.58984375, 1.683168316831683, 1.7611940298507462, 2.5487364620938626, 1.6214285714285714, 1.7256944444444444, 2.491289198606272, 1.6598639455782314, 1.6866197183098592]
#WARNING, len(dividedrankmean)= 35. This is because the very last category(['unknown', 3, 2, 'S']) is empty
    

_lambda = []
_p = []
theta = [] # the rate of expected clicks for ads at rank=r.
bid = [] # what we bid for each type of customer

#fill in values
for i in range(0,35):
    _lambda.append(0) #default 0
    _p.append(0) #default 0
    bid.append(10) #default 10
#estimated values for theta
for i in range(0,5):
    theta.append(clickperrank[i]/totalrank[i])

#estimated values for competitionrate. Assume exponential MLE.
for i in range(0, 35):
    _p[i] = ((dividedrankmean[i]-1)/4)
    _lambda[i] = 10*(dividedrankmean[i]-1)/(5-dividedrankmean[i])


In [3]:
#expectedcost, expectedsold, expected cost per policy sold
def expectedsold(bid):#expected number of policies sold
    for i in range(0, len(bid)):
        if bid[i]<0:
            return 0
    e = 0
    for i in range(0, 35):#35 because click at 36 = 0, lambda will cause errors
        for r in range(1, 6):
            if bid[i]<0:
                e = e +0
            elif bid[i]<10: # when bid is less htan 10, basically just uniform
                e = e + total[i]*probSoldGivenClick[i]*theta[r-1]*math.comb(4, r-1)*(1 - bid[i]*(1-_p[i])/(10))**(r-1)*(bid[i]*(1-_p[i])/(10))**(5-r)
            else: #some aspects of uniform and exponential
                e = e + total[i]*probSoldGivenClick[i]*theta[r-1]*math.comb(4, r-1)*(_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(r-1)*(1-_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(5-r)
    return e
def expectedcost(bid):#expected cost of policy
    e = 0
    for i in range(0, 35):
        for r in range(1, 6):
            if bid[i]<0:
                e = e+0
            elif bid[i]<10: # when bid is less htan 10, basically just uniform
                e = e + total[i]*bid[i]*theta[r-1]*math.comb(4, r-1)*(1 - bid[i]*(1-_p[i])/(10))**(r-1)*(bid[i]*(1-_p[i])/(10))**(5-r)
            else: #some aspects of uniform and exponential
                e = e + total[i]*bid[i]*theta[r-1]*math.comb(4, r-1)*(_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(r-1)*(1-_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(5-r)
        #print(e)
    return e

def costpersold(bid):
    return expectedcost(bid)/expectedsold(bid)
def constraint(bid): #constaining that the expected number of policies sold is more than 400
    return expectedsold(bid)


In [4]:
#hessian is zero matrix if needed
def cons_H(x, v):
    return np.zeros((35,35))
#nonlinar constraint is the constraint function is bounded from 400 to 1000
nonlinear_constraint = NonlinearConstraint(constraint, 400, 1000)#, hess=cons_H)
#linear constraint is each bid is between 1 to 50
lincon = LinearConstraint(np.identity(35), np.linspace(0.01, 0.01, num=35), np.linspace(50000, 50000, num=35))

In [6]:
nonlin =[]
for i in range(0,10):
    nonlin.append(NonlinearConstraint(constraint, 100+i*100, 100+i*100, hess=cons_H))
    
res = []
print(datetime.datetime.now())
for i in range(0,10):
    res.append(minimize(expectedcost, bid, method='trust-constr', constraints=[lincon, nonlin[i]],  options={'maxiter':5000})) 
    print(datetime.datetime.now())
for i in range(0,10):
    print(expectedcost(res[i].x), expectedsold(res[i].x), costpersold(res[i].x))

2021-05-26 22:32:47.832670


/usr/lib/python3/dist-packages/scipy/optimize/_trustregion_constr/projections.py:182: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  warn('Singular Jacobian matrix. Using SVD decomposition to ' +


2021-05-26 22:33:08.832423
2021-05-26 22:33:33.822093


/usr/lib/python3/dist-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


2021-05-26 22:35:03.173280
2021-05-26 22:36:38.285496
2021-05-26 22:39:29.639978
2021-05-26 22:46:22.464571
2021-05-26 22:54:00.274289
2021-05-26 23:04:31.632115
2021-05-26 23:18:42.972054
2021-05-26 23:33:26.648661
65.46589652966658 100.00000000000001 0.6546589652966657
1048.417655632343 200.0 5.242088278161715
2774.5358815882432 300.0 9.248452938627478
5032.491973486614 400.0 12.581229933716536
7710.440684912423 500.00000000000006 15.420881369824846
10775.11267272361 599.9999998547928 17.958521125552192
14307.373361609125 699.9999989847703 20.43910483194216
18390.68097990778 799.9994123706127 22.988368110685556
23237.593562317546 899.9999999999985 25.819548402575094
29076.613194861482 1000.0000000000007 29.076613194861462


In [8]:
currentinsured, numcars, numdrivers, married = [], [],[],[]

for i in range(0, len(_index)):
    currentinsured.append(_index[i][0])
    numcars.append(_index[i][1])
    numdrivers.append(_index[i][2])
    married.append(_index[i][3])

optimizedbids = pd.DataFrame({"Currently Insured":currentinsured[:-1],
                              "Number of Vehicles":numcars[:-1],
                              'Number of Drivers':numdrivers[:-1],
                              'Marital Status':married[:-1],
                             'Bids to get 100 Sales':res[0].x,
                             'Bids to get 200 Sales':res[1].x,
                             'Bids to get 300 Sales':res[2].x,
                             'Bids to get 400 Sales':res[3].x,
                             'Bids to get 500 Sales':res[4].x,
                             'Bids to get 600 Sales':res[5].x,
                             'Bids to get 700 Sales':res[6].x,
                             'Bids to get 800 Sales':res[7].x,
                             'Bids to get 900 Sales':res[8].x,
                             'Bids to get 1000 Sales':res[9].x,})


In [9]:
optimizedbids.to_csv('optimizedbidshybrid.csv')

In [10]:
OptimizedBidsHybrid = pd.DataFrame({"Currently Insured":currentinsured[:-1],
                              "Number of Vehicles":numcars[:-1],
                              'Number of Drivers':numdrivers[:-1],
                              'Marital Status':married[:-1],
                             'Bids to get 400 Sales':res[3].x,})

In [19]:
OptimizedBidsHybrid = OptimizedBidsHybrid.sort_values(by="Bids to get 400 Sales", ascending=False)

OptimizedBidsHybrid.to_csv('OptimizedBidsHybrid.csv')

Optimizing sales while constraining cost

In [13]:
nonlinear_constraint = NonlinearConstraint(expectedcost, 18780, 18780)#, hess=cons_H)
def invsales(bid):
    return 1/expectedsold(bid)

res = minimize(invsales, bid, method='trust-constr', constraints=[lincon, nonlinear_constraint],  options={'maxiter':5000})

In [15]:
res.x

array([ 8.59547983, 11.67992348, 10.92444922,  7.12966982, 11.48712444,
       11.06329907, 10.87752606, 10.68931276, 10.81072844,  7.16150036,
       11.92823184, 12.30532591, 11.76441578, 10.30035572, 11.74033826,
        9.28781398, 12.92267855, 12.40519862, 12.13956407, 10.50969712,
       13.70927409,  9.64373188, 11.76028749, 10.67765957, 11.02552332,
        6.27956963,  8.94376059, 10.30071457, 10.80843588, 10.05969273,
       11.04004793,  9.1848083 ,  8.21139503, 11.08985619,  9.97980051])

In [16]:
print(expectedsold(res.x), expectedcost(res.x), costpersold(res.x))

784.9086618112938 18779.999999985303 23.926350814689258


In [20]:
OptimizedBidsHybrid

,Currently Insured,Number of Vehicles,Number of Drivers,Marital Status,Bids to get 400 Sales
30,unknown,1,1,S,8.119267
33,unknown,1,2,M,8.085248
24,unknown,1,1,M,7.759415
28,unknown,2,1,M,6.980840
18,N,1,1,S,6.865665
6,Y,1,1,S,6.340538
12,N,1,1,M,6.307698
27,unknown,1,2,S,5.942719
34,unknown,2,1,S,5.532367
29,unknown,3,2,M,5.495156


In [21]:
OptimizedCost = pd.DataFrame({"Currently Insured":currentinsured[:-1],
                              "Number of Vehicles":numcars[:-1],
                              'Number of Drivers':numdrivers[:-1],
                              'Marital Status':married[:-1],
                             'Optimized Bids Constraining Cost':res.x,})

In [23]:
OptimizedCost = OptimizedCost.sort_values(by='Optimized Bids Constraining Cost', ascending=False)


In [24]:
OptimizedCost

,Currently Insured,Number of Vehicles,Number of Drivers,Marital Status,Optimized Bids Constraining Cost
20,N,3,1,M,13.709274
16,N,2,1,M,12.922679
17,N,3,2,M,12.405199
11,Y,3,2,S,12.305326
18,N,1,1,S,12.139564
10,Y,2,1,S,11.928232
12,N,1,1,M,11.764416
22,N,2,1,S,11.760287
14,N,3,1,S,11.740338
1,Y,2,2,M,11.679923


In [25]:
OptimizedCost.to_csv('OptimizedCost.csv')